In [7]:
import os, csv, random

from __future__ import division
from matplotlib import  pyplot as plt
from scipy import stats
from statsmodels.base.model import GenericLikelihoodModel
# from pathos.multiprocessing import ProcessingPool as Pool

import numpy as np
import scipy.io as sio
import seaborn as sns

%matplotlib inline

# Generate score for videos, and find all possible pairs

In [8]:
video_num = 30

video_score = np.random.uniform(-5,5,video_num)
resolution = 0.1
video_score = np.round(video_score/resolution)*resolution

pairs_truth = []

for i in range(len(video_score)-1):
    for j in range(i+1, len(video_score)):
        if video_score[i] > video_score[j]:
            pairs_truth.append((i,j))
        else:
            pairs_truth.append((j,i))
            
total_pairs = len(pairs_truth)
print 'Total number of pairs for %d videos is: ' %video_num, total_pairs

Total number of pairs for 30 videos is:  435


# SGD
Assuming the pair $(w_i,w_j)$ means $w_i > w_j$, this function calculates $\frac{\partial L}{\partial w_i} = \frac{exp(w_j-w_i)}{1+exp(w_j-w_i)}$, where $w_i$ represent the weight to be taken derivative of. Return value is the gradient of the log likelihood function with respect to $w_i$

In [9]:
def calc_gradient(pairs, w, w_i):
    gradient = 0
    
    for pair in pairs:
        if w_i == pair[0]:
            out = -1
        elif w_i == pair[1]:
            out = 1  
        else:
            out = 0
            
        gradient -= out/(1/(np.exp(w[pair[1]]-w[pair[0]]))+1)
    
    return gradient

$w_{old}$ saves the original values of the gradient, so that later when updating the values, the values to be updated are still using the original value

In [10]:
def update_gradient(pairs, w, eta):
    w_old = w
    for i in range(len(w)):
        w[i] += eta*calc_gradient(pairs, w_old, i)
    return w

In [11]:
def likelihood(pairs, w):
    l = 1
    for pair in pairs:
        l += np.log(1+np.exp(w[pair[1]]-w[pair[0]]))
    return l

In [ ]:
w = np.zeros(30)
pairs = pairs_truth
results = []
for i in range(15000):
    
    w = update_gradient(pairs, w, 0.005)
    if i % 1000 ==0:
        print likelihood(pairs_truth, w)
    results.append(w[0])
    

291.498950583
35.9021508409


In [ ]:
true_order = np.array(video_score).argsort()
true_ranks = true_order.argsort()

temp_o = np.array(w).argsort()
temp_r = temp_o.argsort()

print true_ranks
print temp_r
print w

# Generate Data using Generative function 

In [ ]:
video_score

In [7]:
from multiprocessing import Pool


def f(i):
    print 'Evaluating trait %d', i+1
    model = Pairwise(pairs[i])
    w_init = np.random.uniform(-5,5,video_num)
    results = model.fit(w_init, maxiter=1000000, maxfun=1000000)
    estimated_for_each_trait[str(i)] = results.params
    return estimated_for_each_trait

def mp_handler():
    p = multiprocessing.Pool(2)
    p.map(f, [0,1,2,3,4])
    
mp_handler()

NameError: global name 'multiprocessing' is not defined

In [8]:
def update_gradient_concur(pairs, w):
    w_old = w
    eta = 0.01
    for i in range(len(w)):
        w[i] += eta*calc_gradient(pairs, w_old, i)
    return w

In [ ]:
import multiprocessing

w = np.zeros(30)
pairs = pairs_truth

p = multiprocessing.Pool(2)

for i in range(10000):
    w = p.map(update_gradient_concur, pairs, w)
    if i % 500 ==0:
        print likelihood(pairs_truth, w)
        
    


    p = multiprocessing.Process(target=update_gradient_concur, args=(pairs, w,))
    p.start()
    p.join()

In [41]:
def update_gradient_concur(pairs, w):
    import numpy as np
#     print pairs # result is a list of pairs
    w_old = w[0]
    w = w[0]
    eta = 0.01
    for i in range(len(w)):
        gradient = 0
    
        for pair in pairs:
            if i == pair[0]:
                out = -1
            elif i == pair[1]:
                out = 1  
            else:
                out = 0
            
        gradient -= out/(1/(np.exp(w_old[pair[1]]-w_old[pair[0]]))+1)
        
        w[i] += eta*gradient
    return w

def likelihood(pairs, w):
    l = 1
    w = w[0]
    for pair in pairs:        
        l += np.log(1+np.exp(w[pair[1]]-w[pair[0]]))
    return l

w = np.zeros(30)

for i in range(10000):
    w = Pool().map(update_gradient_concur, [pairs_truth], [w])
    if i % 500 ==0:
        print likelihood(pairs_truth, w)

print res

TypeError: object of type 'numpy.float64' has no len()